# 1c) Check whether a tweet is harmful to society

In [1]:
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()

## Data exploration

In [2]:
df_test = pd.read_csv('data/1c/CT22_english_1C_harmful_dev_test.tsv', sep='\t')
df_train = pd.read_csv('data/1c/CT22_english_1C_harmful_train.tsv', sep='\t')
df = pd.read_csv('data/1c/CT22_english_1C_harmful_dev.tsv', sep='\t')

print("Test shape: {}\nTrain shape: {}\nNormal shape: {}".format(df_test.shape, df_train.shape, df.shape))

df_train.head()

Test shape: (910, 5)
Train shape: (3323, 5)
Normal shape: (307, 5)


,topic,tweet_id,tweet_url,tweet_text,class_label
0,COVID-19,1.359351e+18,http://twitter.com/user/status/135935094335617...,"India's gift of 100,000 COVID-19 vaccines arri...",0
1,COVID-19,1.350166e+18,http://twitter.com/user/status/135016568806166...,Here’s what I’m doing while I wait my turn for...,0
2,COVID-19,1.369750e+18,http://twitter.com/user/status/136974953915491...,"This afternoon, I’m hosting an event with the ...",0
3,COVID-19,1.350165e+18,http://twitter.com/user/status/135016499568693...,"Help shops like mine stay open. Mask up, avoid...",0
4,COVID-19,1.370008e+18,http://twitter.com/user/status/137000807648978...,As part of the ongoing nationwide vaccination ...,0


In [3]:
# Train set unique
print(df_train['topic'].unique())
print(df_train['class_label'].unique())

# Test set unique
print(df_test['topic'].unique())
print(df_test['class_label'].unique())

# Normal
print(df['topic'].unique())
print(df['class_label'].unique())

['COVID-19']
[0 1]
['COVID-19']
[0 1]
['COVID-19']
[0 1]


## Cleanup

In [4]:
tweets_list = df_train['tweet_text'].to_list()

In [5]:
# Removing stop words from: https://www.geeksforgeeks.org/removing-stop-words-nltk-python/

def preprocess(tweets):
    processed_list = []

    # URL PATTERN copied from: https://stackoverflow.com/questions/520031/whats-the-cleanest-way-to-extract-urls-from-a-string-using-python
    URL_PATTERN = r"""(?i)\b((?:https?:(?:/{1,3}|[a-z0-9%])|[a-z0-9.\-]+[.](?:com|net|org|edu|gov|mil|aero|asia|biz|cat|coop|info|int|jobs|mobi|museum|name|post|pro|tel|travel|xxx|ac|ad|ae|af|ag|ai|al|am|an|ao|aq|ar|as|at|au|aw|ax|az|ba|bb|bd|be|bf|bg|bh|bi|bj|bm|bn|bo|br|bs|bt|bv|bw|by|bz|ca|cc|cd|cf|cg|ch|ci|ck|cl|cm|cn|co|cr|cs|cu|cv|cx|cy|cz|dd|de|dj|dk|dm|do|dz|ec|ee|eg|eh|er|es|et|eu|fi|fj|fk|fm|fo|fr|ga|gb|gd|ge|gf|gg|gh|gi|gl|gm|gn|gp|gq|gr|gs|gt|gu|gw|gy|hk|hm|hn|hr|ht|hu|id|ie|il|im|in|io|iq|ir|is|it|je|jm|jo|jp|ke|kg|kh|ki|km|kn|kp|kr|kw|ky|kz|la|lb|lc|li|lk|lr|ls|lt|lu|lv|ly|ma|mc|md|me|mg|mh|mk|ml|mm|mn|mo|mp|mq|mr|ms|mt|mu|mv|mw|mx|my|mz|na|nc|ne|nf|ng|ni|nl|no|np|nr|nu|nz|om|pa|pe|pf|pg|ph|pk|pl|pm|pn|pr|ps|pt|pw|py|qa|re|ro|rs|ru|rw|sa|sb|sc|sd|se|sg|sh|si|sj|Ja|sk|sl|sm|sn|so|sr|ss|st|su|sv|sx|sy|sz|tc|td|tf|tg|th|tj|tk|tl|tm|tn|to|tp|tr|tt|tv|tw|tz|ua|ug|uk|us|uy|uz|va|vc|ve|vg|vi|vn|vu|wf|ws|ye|yt|yu|za|zm|zw)/)(?:[^\s()<>{}\[\]]+|\([^\s()]*?\([^\s()]+\)[^\s()]*?\)|\([^\s]+?\))+(?:\([^\s()]*?\([^\s()]+\)[^\s()]*?\)|\([^\s]+?\)|[^\s`!()\[\]{};:\'\".,<>?«»“”‘’])|(?:(?<!@)[a-z0-9]+(?:[.\-][a-z0-9]+)*[.](?:com|net|org|edu|gov|mil|aero|asia|biz|cat|coop|info|int|jobs|mobi|museum|name|post|pro|tel|travel|xxx|ac|ad|ae|af|ag|ai|al|am|an|ao|aq|ar|as|at|au|aw|ax|az|ba|bb|bd|be|bf|bg|bh|bi|bj|bm|bn|bo|br|bs|bt|bv|bw|by|bz|ca|cc|cd|cf|cg|ch|ci|ck|cl|cm|cn|co|cr|cs|cu|cv|cx|cy|cz|dd|de|dj|dk|dm|do|dz|ec|ee|eg|eh|er|es|et|eu|fi|fj|fk|fm|fo|fr|ga|gb|gd|ge|gf|gg|gh|gi|gl|gm|gn|gp|gq|gr|gs|gt|gu|gw|gy|hk|hm|hn|hr|ht|hu|id|ie|il|im|in|io|iq|ir|is|it|je|jm|jo|jp|ke|kg|kh|ki|km|kn|kp|kr|kw|ky|kz|la|lb|lc|li|lk|lr|ls|lt|lu|lv|ly|ma|mc|md|me|mg|mh|mk|ml|mm|mn|mo|mp|mq|mr|ms|mt|mu|mv|mw|mx|my|mz|na|nc|ne|nf|ng|ni|nl|no|np|nr|nu|nz|om|pa|pe|pf|pg|ph|pk|pl|pm|pn|pr|ps|pt|pw|py|qa|re|ro|rs|ru|rw|sa|sb|sc|sd|se|sg|sh|si|sj|Ja|sk|sl|sm|sn|so|sr|ss|st|su|sv|sx|sy|sz|tc|td|tf|tg|th|tj|tk|tl|tm|tn|to|tp|tr|tt|tv|tw|tz|ua|ug|uk|us|uy|uz|va|vc|ve|vg|vi|vn|vu|wf|ws|ye|yt|yu|za|zm|zw)\b/?(?!@)))"""
    STOP_WORDS = set(stopwords.words('english'))

    
    for tweet in tweets:
        tweet = re.sub(URL_PATTERN, 'URL', tweet)
        tweet = tweet.lower()
        tweet = re.sub('\d', '', tweet) # Remove digits
        tweet = re.sub('[^A-Za-z0-9]+', ' ', tweet) # Remove all special charaters

        # Remove stopwords
        tokens = word_tokenize(tweet)
        tweet = [word for word in tokens if not word in STOP_WORDS]
        tweet = "".join(tweet)
        processed_list.append(tweet)
    return processed_list

tweets_train = preprocess(df_train['tweet_text'].to_list())
tweets_test = preprocess(df_test['tweet_text'].to_list())

# Training the algorithmn

In [12]:
train_vocabulary = vectorizer.fit(tweets_train)

y_train = df_train['class_label'].to_list()
y_test = df_test['class_label'].to_list()

X_test = train_vocabulary.transform(tweets_test)
X_train = train_vocabulary.transform(tweets_train)


In [11]:
gnb = GaussianNB()
y_pred = gnb.fit(X_train.toarray(), y_train).predict(X_test.toarray())
print("Number of mislabeled points out of a total %d points : %d"% (X_test.shape[0], (y_test != y_pred).sum()))

Number of mislabeled points out of a total 910 points : 827


# (USE F1 FOR EVALUATION)